## Import data from the source files

--> Data from https://github.com/krlawrence/graph/raw/master/sample-data/ is available under the Apache 2.0 license. Courtesy of Kelvin Lawrence.

In [1]:
import pandas as pd

df_nodes=pd.read_csv('https://github.com/krlawrence/graph/raw/master/sample-data/air-routes-latest-nodes.csv')
df_edges=pd.read_csv('https://github.com/krlawrence/graph/raw/master/sample-data/air-routes-latest-edges.csv')

Preview a Pandas DataFrame with imported data

In [2]:
df_nodes.head(3)

,~id,~label,type:string,code:string,icao:string,desc:string,region:string,runways:int,longest:int,elev:int,country:string,city:string,lat:double,lon:double,author:string,date:string
0,0,version,version,0.89,NaN,Air Routes Data - Version: 0.89 Generated: 202...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Kelvin R. Lawrence,2022-08-29 14:10:18 UTC
1,1,airport,airport,ATL,KATL,Hartsfield - Jackson Atlanta International Air...,US-GA,5.0,12390.0,1026.0,US,Atlanta,33.6367,-84.428101,NaN,NaN
2,2,airport,airport,ANC,PANC,Anchorage Ted Stevens,US-AK,3.0,12400.0,151.0,US,Anchorage,61.1744,-149.996002,NaN,NaN


### Create a Pandas DataFrame (`df_ports`) with airports only

In [3]:
df_nodes.dtypes

~id                 int64
~label             object
type:string        object
code:string        object
icao:string        object
desc:string        object
region:string      object
runways:int       float64
longest:int       float64
elev:int          float64
country:string     object
city:string        object
lat:double        float64
lon:double        float64
author:string      object
date:string        object
dtype: object

The DataFrame contains different types of data that are using different `~label` values:

In [4]:
df_nodes.groupby('~label').size()

~label
airport      3504
continent       7
country       237
version         1
dtype: int64

1. Keep only records with `airport` labels.
2. Remove unnecessary columns.

In [5]:
df_ports=(
           df_nodes[df_nodes['~label'].isin(['airport'])]
           .drop(['~label','type:string','author:string','date:string'], axis=1)
           .convert_dtypes()
          )

In [6]:
print(df_ports.dtypes)

~id                        Int64
code:string       string[python]
icao:string       string[python]
desc:string       string[python]
region:string     string[python]
runways:int                Int64
longest:int                Int64
elev:int                   Int64
country:string    string[python]
city:string       string[python]
lat:double               Float64
lon:double               Float64
dtype: object


Clean up the column names

In [7]:
df_ports.columns=(df_ports.columns
                   .str.replace('~','')
                   .str.split(':').str[0]
                   .str.upper()
                  )

In [8]:
df_ports.dtypes

ID                  Int64
CODE       string[python]
ICAO       string[python]
DESC       string[python]
REGION     string[python]
RUNWAYS             Int64
LONGEST             Int64
ELEV                Int64
COUNTRY    string[python]
CITY       string[python]
LAT               Float64
LON               Float64
dtype: object

### Create a Pandas DataFrame (`df_routes`) with connections between the airports only

In [9]:
df_edges.dtypes

~id           int64
~from         int64
~to           int64
~label       object
dist:int    float64
dtype: object

In [10]:
df_edges.groupby('~label').size()

~label
contains     7008
route       50637
dtype: int64

1. Keep only records with `route` labels.
2. Remove unnecessary column `~label`.

In [11]:
df_routes=df_edges[df_edges['~label'].isin(['route'])].drop(['~label'], axis=1).convert_dtypes()

Clean up the column names

In [12]:
df_routes.columns=df_routes.columns.str.replace('~','').str.split(':').str[0].str.upper()

In [13]:
df_routes.dtypes

ID      Int64
FROM    Int64
TO      Int64
DIST    Int64
dtype: object

## Upload into your SAP HANA database

In [14]:
import os, hana_ml
print(hana_ml.__version__)

2.21.24062400


In [15]:
os.environ["HDB_USE_IDENT"]=os.getenv("WORKSPACE_ID")
print(os.getenv("HDB_USE_IDENT"))

workspaces-ws-vklhr


In [16]:
from hana_ml import dataframe as hdf

In [17]:
myconn=hdf.ConnectionContext(userkey='myDevChallenger')
print("SAP HANA DB version: ", myconn.hana_version())

SAP HANA DB version:  4.00.000.00.1718710788 (fa/CE2024.14)


Upload data from a Pandas DataFrame to a SAP HANA database table and return an SAP HANA DataFrame `hdf_*`: https://help.sap.com/doc/cd94b08fe2e041c2ba778374572ddba9/2024_2_QRC/en-US/hana_ml.dataframe.html#hana_ml.dataframe.create_dataframe_from_pandas

In [18]:
hdf_ports=hdf.create_dataframe_from_pandas(
    connection_context=myconn,
    pandas_df=df_ports,
    table_name="PORTS",
    force=True
)

100%|██████████| 1/1 [00:00<00:00, 16.25it/s]


In [19]:
hdf_routes=hdf.create_dataframe_from_pandas(
    connection_context=myconn, 
    pandas_df=df_routes, 
    table_name='ROUTES',
    force=True
)

100%|██████████| 2/2 [00:00<00:00,  8.38it/s]


### Data exploration using HANA DataFrames

Return a dictionary format of a table structure: https://help.sap.com/doc/cd94b08fe2e041c2ba778374572ddba9/2024_2_QRC/en-US/hana_ml.dataframe.html#hana_ml.dataframe.DataFrame.get_table_structure

In [20]:
hdf_ports.get_table_structure()

{'ID': 'INT',
 'CODE': 'NVARCHAR(5000)',
 'ICAO': 'NVARCHAR(5000)',
 'DESC': 'NVARCHAR(5000)',
 'REGION': 'NVARCHAR(5000)',
 'RUNWAYS': 'INT',
 'LONGEST': 'INT',
 'ELEV': 'INT',
 'COUNTRY': 'NVARCHAR(5000)',
 'CITY': 'NVARCHAR(5000)',
 'LAT': 'DOUBLE',
 'LON': 'DOUBLE'}

**What is the airport with the longest runway?**

Note the use of:
- [select()](https://help.sap.com/doc/cd94b08fe2e041c2ba778374572ddba9/2024_2_QRC/en-US/hana_ml.dataframe.html#hana_ml.dataframe.DataFrame.select)
- [sort()](https://help.sap.com/doc/cd94b08fe2e041c2ba778374572ddba9/2024_2_QRC/en-US/hana_ml.dataframe.html#hana_ml.dataframe.DataFrame.sort)
- [head()](https://help.sap.com/doc/cd94b08fe2e041c2ba778374572ddba9/2024_2_QRC/en-US/hana_ml.dataframe.html#hana_ml.dataframe.DataFrame.head)
- [collect()](https://help.sap.com/doc/cd94b08fe2e041c2ba778374572ddba9/2024_2_QRC/en-US/hana_ml.dataframe.html#hana_ml.dataframe.DataFrame.collect)

In [21]:
(
    hdf_ports
    .select("CODE", "DESC", "LONGEST", "COUNTRY", "CITY")
    .sort("LONGEST", desc=True)
    .head().collect()
)

,CODE,DESC,LONGEST,COUNTRY,CITY
0,BPX,Qamdo Bangda Airport,18045,CN,Bangda


**What country has an airport with the highest number of runways?**

Note the use of:
- [agg()](https://help.sap.com/doc/cd94b08fe2e041c2ba778374572ddba9/2024_2_QRC/en-US/hana_ml.dataframe.html#hana_ml.dataframe.DataFrame.agg)

In [22]:
(
    hdf_ports
    .agg(
        agg_list=[("max", "RUNWAYS", "MAXRUNWAYS")], 
        group_by="COUNTRY"
    )
    .sort("MAXRUNWAYS", desc=True)
    .head().collect()
)

,COUNTRY,MAXRUNWAYS
0,US,7


**What is the airport closest to either the North or South Pole?**

Note the use of the calculated column `ABSOLUTE_LATITUDE` in a `select()`

In [23]:
(
    hdf_ports
    .select(
        "CODE", "DESC", "COUNTRY", "CITY", "LAT", "LON",
        ('ABS("LAT")', "ABSOLUTE_LATITUDE")
    )
    .sort("ABSOLUTE_LATITUDE", desc=True).head()
    .collect()
)

,CODE,DESC,COUNTRY,CITY,LAT,LON,ABSOLUTE_LATITUDE
0,LYR,"Svalbard Airport, Longyear",NO,Longyearbyen,78.246101,15.4656,78.246101


**How far are the 3 southernmost airports from the South Pole?**

Note the use of:
- constructor [`ST_Point()`](https://help.sap.com/docs/hana-cloud-database/sap-hana-cloud-sap-hana-database-spatial-reference/st-point-double-double-integer-constructor?version=2024_2_QRC&locale=en-US)
- method [`ST_Distance()`](https://help.sap.com/docs/hana-cloud-database/sap-hana-cloud-sap-hana-database-spatial-reference/st-distance-method)
- [Spatial Reference Identifier (SRID)](https://help.sap.com/docs/hana-cloud-database/sap-hana-cloud-sap-hana-database-spatial-reference/spatial-reference-systems-srs-and-spatial-reference-identifiers-srid) `4326` to make points and calculations on the Round Earth, and not a 2D projection
- [unit of measure](https://help.sap.com/docs/hana-cloud-database/sap-hana-cloud-sap-hana-database-spatial-reference/units-of-measure) for the distance result

In [24]:
SRID=4326
UNIT_OF_MEASURE='kilometer'

(
    hdf_ports
    .select(
        "CODE", "DESC", "COUNTRY", "CITY", "LAT", "LON", 
        (f'''NEW ST_Point("LON", -90, {SRID}).ST_Distance(NEW ST_Point("LON", "LAT", {SRID}), '{UNIT_OF_MEASURE}')''', f"DISTANCE_FROM_SOUTHPOLE_IN_{UNIT_OF_MEASURE}")
    )
    .sort(f"DISTANCE_FROM_SOUTHPOLE_IN_{UNIT_OF_MEASURE}", desc=False).head(3)
    .collect()
)

,CODE,DESC,COUNTRY,CITY,LAT,LON,DISTANCE_FROM_SOUTHPOLE_IN_kilometer
0,USH,Malvinas Argentinas Airport,AR,Ushuahia,-54.843300,-68.295800,3922.179587
1,RGA,Hermes Quijada International Airport,AR,Rio Grande,-53.777700,-67.749400,4040.792342
2,PUQ,Pdte. Carlos Ibañez del Campo Airport,CL,Punta Arenas,-53.002602,-70.854599,4127.055828


👉 **Spatial units of measure** that can be used in queries are listed in the [system view `ST_UNITS_OF_MEASURE`](https://help.sap.com/docs/hana-cloud-database/sap-hana-cloud-sap-hana-database-sql-reference-guide/st-units-of-measure-system-view?version=2024_2_QRC&locale=en-US).

In the query above you used the `kilometer`, but you can try some other units as well.

In [25]:
myconn.table("ST_UNITS_OF_MEASURE", schema="PUBLIC").collect()

,OWNER_NAME,UNIT_NAME,UNIT_TYPE,CONVERSION_FACTOR,CREATOR
0,SYS,50_Kilometers,LINEAR,50000.000000,SYS
1,SYS,kilometer,LINEAR,1000.000000,SYS
2,SYS,Clarke's yard,LINEAR,0.914392,SYS
3,SYS,degree,ANGULAR,0.017453,SYS
4,SYS,British chain (Benoit 1895 B),LINEAR,20.116782,SYS
5,SYS,link,LINEAR,0.201168,SYS
6,SYS,German legal metre,LINEAR,1.000014,SYS
7,SYS,grad,ANGULAR,0.015708,SYS
8,SYS,British chain (Sears 1922 truncated),LINEAR,20.116756,SYS
9,SYS,metre,LINEAR,1.000000,SYS


👉 **Spatial reference systems** that can be used in queries are listed in the [system view `ST_SPATIAL_REFERENCE_SYSTEMS`](https://help.sap.com/docs/hana-cloud-database/sap-hana-cloud-sap-hana-database-sql-reference-guide/st-spatial-reference-systems-system-view?version=2024_2_QRC&locale=en-US).

In the query above you used `4326`, which is the spatial reference system id (or **SRID**) standard used in cartography, geodesy, and satellite navigation including GPS: https://epsg.org/crs_4326/WGS-84.html.

In [26]:
myconn.table("ST_SPATIAL_REFERENCE_SYSTEMS", schema="PUBLIC").collect()

,OWNER_NAME,SRS_ID,SRS_NAME,ROUND_EARTH,AXIS_ORDER,SNAP_TO_GRID,TOLERANCE,SEMI_MAJOR_AXIS,SEMI_MINOR_AXIS,INV_FLATTENING,...,ORGANIZATION,ORGANIZATION_COORDSYS_ID,SRS_TYPE,LINEAR_UNIT_OF_MEASURE,ANGULAR_UNIT_OF_MEASURE,POLYGON_FORMAT,STORAGE_FORMAT,DEFINITION,TRANSFORM_DEFINITION,CREATOR
0,SYS,0,Default,FALSE,x/y/z/m,1.000000e-06,0.0,NaN,None,NaN,...,SAP,0,ENGINEERING,meter,None,EVENODD,INTERNAL,None,None,SYS
1,SYS,3857,WGS 84 / Pseudo-Mercator,FALSE,x/y/z/m,1.000000e-04,0.0,NaN,None,NaN,...,EPSG,3857,PROJECTED,metre,degree,EVENODD,INTERNAL,"PROJCS[""WGS 84 / Pseudo-Mercator"",GEOGCS[""WGS ...",+proj=merc +a=6378137 +b=6378137 +lat_ts=0 +lo...,SYS
2,SYS,4326,WGS 84,TRUE,long/lat/z/m,0.000000e+00,0.0,6378137.0,None,298.257224,...,EPSG,4326,GEOGRAPHIC,metre,degree,EVENODD,MIXED,"GEOGCS[""WGS 84"",DATUM[""WGS_1984"",SPHEROID[""WGS...",+proj=longlat +datum=WGS84 +no_defs,SYS
3,SYS,1000004326,WGS 84 (planar),FALSE,long/lat/z/m,1.000000e-09,0.0,6378137.0,None,298.257224,...,EPSG,4326,PROJECTED,planar degree,degree,EVENODD,INTERNAL,"GEOGCS[""WGS 84"",DATUM[""WGS_1984"",SPHEROID[""WGS...",+proj=longlat +datum=WGS84 +no_defs,SYS
4,SYS,2147483646,Unbounded (planar),FALSE,x/y/z/m,0.000000e+00,0.0,NaN,None,NaN,...,SAP,2147483646,ENGINEERING,meter,None,EVENODD,INTERNAL,None,None,SYS


**What are the two closest airport?**

Note the use of:
- [join()](https://help.sap.com/doc/cd94b08fe2e041c2ba778374572ddba9/2024_2_QRC/en-US/hana_ml.dataframe.html#hana_ml.dataframe.DataFrame.join)

In [27]:
(
    hdf_routes
    .sort("DIST", desc=False)
    .head()
    .collect()
)

,ID,FROM,TO,DIST
0,49420,1935,1932,2


In [28]:
(
    hdf_routes.sort("DIST", desc=False).head()
    .alias('L1').join(hdf_ports.select(("ID", "FROM_ID"), "ICAO", "DESC").alias('R1'), 'L1."FROM" = R1."FROM_ID"')
    .alias('L2').join(hdf_ports.select(("ID", "TO_ID"), "ICAO", "DESC").alias('R2'), 'L2."TO" = R2."TO_ID"')
    .collect().iloc[:, 3:]
)

,DIST,FROM_ID,ICAO,DESC,TO_ID,ICAO,DESC
0,2,1935,EGEW,Westray Airport,1932,EGEP,Papa Westray Airport


Closer inspection on the map explains such a short distance between the two airports: https://en.mapy.cz/zakladni?l=0&x=-2.9290799&y=59.3518237&z=14

## Create SAP HANA graph workspace

Use [`hana_ml.graph`](https://help.sap.com/doc/cd94b08fe2e041c2ba778374572ddba9/2024_2_QRC/en-US/hana_ml.graph.html#module-hana_ml.graph) from the Python Machine Learning Client for SAP HANA:
* [create_graph_from_dataframes()](https://help.sap.com/doc/cd94b08fe2e041c2ba778374572ddba9/2024_2_QRC/en-US/hana_ml.graph.html#hana_ml.graph.create_graph_from_dataframes) to model a [graph workspace](https://help.sap.com/docs/hana-cloud-database/sap-hana-cloud-sap-hana-database-graph-reference/sap-hana-graph-data-model)
* [discover_graph_workspaces()](https://help.sap.com/doc/cd94b08fe2e041c2ba778374572ddba9/2024_2_QRC/en-US/hana_ml.graph.html#hana_ml.graph.discover_graph_workspaces) to check existing [graphs workspace artifacts](https://help.sap.com/docs/hana-cloud-database/sap-hana-cloud-sap-hana-database-graph-reference/graph-metadata-views) in SAP HANA database

In [29]:
import hana_ml.graph

In [30]:
hgws_airroutes = (
    hana_ml.graph.create_graph_from_dataframes(
        connection_context=myconn, 
        workspace_name='AIRROUTES_DFH',
        
        vertices_df=hdf_ports,
        vertex_key_column="ID", 
        
        edges_df=hdf_routes, 
        edge_key_column="ID",
        edge_source_column="FROM", edge_target_column="TO"
    )
)

In [31]:
hana_ml.graph.discover_graph_workspaces(myconn)

,SCHEMA_NAME,WORKSPACE_NAME,IS_VALID,EDGE_TARGET_COLUMN,EDGE_SOURCE_COLUMN_NAME,EDGE_SCHEMA_NAME,EDGE_TABLE_NAME,EDGE_KEY_COLUMN_NAME,VERTEX_SCHEMA_NAME,VERTEX_TABLE_NAME,VERTEX_KEY_COLUMN_NAME
0,DEVCHALLENGER,AIRROUTES_DFH,TRUE,TO,FROM,DEVCHALLENGER,ROUTES_GE_VIEW,ID,DEVCHALLENGER,PORTS_GV_VIEW,ID


### Exploring the graph's...

...[vertices](https://help.sap.com/doc/cd94b08fe2e041c2ba778374572ddba9/2024_2_QRC/en-US/hana_ml.graph.html#hana_ml.graph.Graph.vertices) (nodes):

In [32]:
hgws_airroutes.vertices(vertex_key=313)

,ID,CODE,ICAO,DESC,REGION,RUNWAYS,LONGEST,ELEV,COUNTRY,CITY,LAT,LON
0,313,WRO,EPWR,Copernicus Wroclaw Airport,PL-DS,1,8202,404,PL,Wroclaw,51.102699,16.885799


...[edges](https://help.sap.com/doc/cd94b08fe2e041c2ba778374572ddba9/2024_2_QRC/en-US/hana_ml.graph.html#hana_ml.graph.Graph.edges) (connections):

In [33]:
hgws_airroutes.edges(vertex_key=313, direction='INCOMING').head(5)

,ID,FROM,TO,DIST
0,9766,51,313,651
1,10066,52,313,373
2,11296,60,313,987
3,12678,70,313,525
4,13416,74,313,1220
